In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.metrics import pairwise_distances

from pp_mix.simulation.conditional_mh import ConditionalMH
from pp_mix.simulation.birth_death import BirthDeathMH, SpatialBirthAndDeath
from pp_mix.simulation.cftp import SimpleDominatedCftp

ModuleNotFoundError: No module named 'jax'

# Conditional MH

### Poisson Process on [0, 1]

In [2]:
def papangelou(csi, x, log=True):
    out = np.sum(stats.beta.logpdf(csi, 0.5, 0.5))
    if not log:
        out = np.exp(out)
    return out


def proposal_rng(state, ind):
    return stats.multivariate_normal.rvs(mean=state[ind, :], cov=0.1*np.eye(2))


def proposal_dens(x, prev_state, ind, log=True):
    out = stats.multivariate_normal.logpdf(
        x[ind], mean=prev_state[ind, :], cov=0.1*np.eye(2))
    if not log:
        out = np.exp(out)
    return out

In [ ]:
npoints = 30
init_state = np.random.uniform(size=npoints*2).reshape(npoints, 2)

In [ ]:
mh = ConditionalMH(npoints, papangelou, proposal_rng, proposal_dens)
chain = mh.run(1000, 1000, init_state)

In [ ]:
ind = 10
plt.scatter(chain[ind, :, 0], chain[ind, :, 1])

### Strauss process

In [3]:
gamma = 0.1
R = 0.3
scale = 0.2


def papangelou(csi, x, log=True):
    dists = pairwise_distances(x, csi.reshape(1, -1))
    out = np.log(gamma) * np.sum(dists <= R)
    if not log:
        out = np.exp(out)
    return out

def proposal_rng(state, ind):
    mean = state[ind, :]
    a = (0 - mean) / scale
    b = (1 - mean) / scale

    return stats.truncnorm.rvs(a, b, loc=mean, scale=scale)


def proposal_dens(x, prev_state, ind, log=True):
    mean = prev_state[ind, :]
    a = (0 - mean) / scale
    b = (1 - mean) / scale
    
    out = np.sum(stats.truncnorm.logpdf(x[ind, :], a, b, loc=mean, scale=scale))
    if not log:
        out = np.exp(out)
    return out

In [ ]:
mh = ConditionalMH(npoints, papangelou, proposal_rng, proposal_dens)
chain = mh.run(1000, 1000, init_state)

In [ ]:
ind = 905
plt.scatter(chain[ind, :, 0], chain[ind, :, 1])

# Birth and Death MH

In [ ]:
import jax.numpy as np
from jax import random

class Rng(object):
    def __init__(self):
        self.key = random.PRNGKey(0)
    
    
    def get(self):
        _, self.key = random.split(self.key)
        return self.key
    

rng = Rng()

### Poisson Process

In [ ]:
def papangelou(csi, x, log=True):
    out = np.sum(stats.beta.logpdf(csi, 0.5, 0.5))
    if not log:
        out = np.exp(out)
    return out


def update_proposal_rng(state, ind):
    return stats.multivariate_normal.rvs(mean=state[ind, :], cov=0.1*np.eye(2))

def update_proposal_dens(x, prev_state, ind, log=True):
    out = stats.multivariate_normal.logpdf(
        x[ind], mean=prev_state[ind, :], cov=0.1*np.eye(2))
    if not log:
        out = np.exp(out)
    return out


def birth_proposal_rng(state):
    mean = np.array([0.5, 0.5])
    a = (0 - mean) / 0.5
    b = (1 - mean) / 0.5

    return stats.truncnorm.rvs(a, b, loc=mean, scale=0.5)


def birth_proposal_dens(x, state, log=True):
    mean = np.array([0.5, 0.5])
    a = (0 - mean) / 0.5
    b = (1 - mean) / 0.5
    
    out = np.sum(stats.truncnorm.logpdf(x, a, b, loc=mean, scale=0.5))
    if not log:
        out = np.exp(out)
    return out

In [ ]:
npoints = 100
init_state = np.random.uniform(size=npoints*2).reshape(npoints, 2)

In [ ]:
bdmh = BirthDeathMH(papangelou, birth_proposal_rng, birth_proposal_dens,
                    update_proposal_rng, update_proposal_dens)

chains = bdmh.run(rng, 1000, 1000, init_state)

In [ ]:
index = 900
plt.scatter(chains[index][:, 0], chains[index][:, 1])

## Strauss Process

In [ ]:
gamma = 0.1
R = 0.3
scale = 0.2


def papangelou(csi, x, log=True):
    dists = pairwise_distances(x, csi.reshape(1, -1))
    out = np.log(gamma) * np.sum(dists <= R)
    if not log:
        out = np.exp(out)
    return out

def update_proposal_rng(state, ind):
    mean = state[ind, :]
    a = (0 - mean) / scale
    b = (1 - mean) / scale

    return stats.truncnorm.rvs(a, b, loc=mean, scale=scale)


def update_proposal_dens(x, prev_state, ind, log=True):
    mean = prev_state[ind, :]
    a = (0 - mean) / scale
    b = (1 - mean) / scale
    
    out = np.sum(stats.truncnorm.logpdf(x[ind, :], a, b, loc=mean, scale=scale))
    if not log:
        out = np.exp(out)
    return out


def birth_proposal_rng(state):
    mean = np.array([0.5, 0.5])
    a = (0 - mean) / 0.5
    b = (1 - mean) / 0.5

    return stats.truncnorm.rvs(a, b, loc=mean, scale=0.5)


def birth_proposal_dens(x, state, log=True):
    mean = np.array([0.5, 0.5])
    a = (0 - mean) / 0.5
    b = (1 - mean) / 0.5
    
    out = np.sum(stats.truncnorm.logpdf(x, a, b, loc=mean, scale=0.5))
    if not log:
        out = np.exp(out)
    return out

In [ ]:
npoints = 100
init_state = random.uniform(rng.get(), (npoints*2,)).reshape(npoints, 2)

In [ ]:
bdmh = BirthDeathMH(papangelou, birth_proposal_rng, birth_proposal_dens,
                    update_proposal_rng, update_proposal_dens)

chains = bdmh.run(rng, 10000, 10000, init_state)

In [ ]:
index = 8296
plt.scatter(chains[index][:, 0], chains[index][:, 1])

In [ ]:
nums = np.array([x.shape[0] for x in chains])
plt.hist(nums)
plt.show()

# Spatial Birth and Death

In [ ]:
beta = 200
gamma = 0.9
R = 0.5
scale = 0.2


def papangelou(csi, x, log=True):
    if x.shape[0] == 0:
        out = 0
    else:
        dists = pairwise_distances(x, csi.reshape(1, -1))
        out = np.log(gamma) * np.sum(dists <= R)
    if not log:
        out = np.exp(out)
    return out

def phi_star_rng():
    return np.random.uniform(size=2)

def phi_star_dens(csi, log=True):
    out = 0;
    if not log:
        out = np.exp(out)
    return out

def dens(x, log=True):
    dists = pairwise_distances(x)
    out = np.sum(np.log(beta) * x.shape[1])
    out += np.log(gamma) * np.sum(dists <= R)
    if not log:
        out = np.exp(out)
    return out

c_star = beta

In [ ]:
npoints = 100
init_state = random.uniform(rng.get(), (npoints*2,)).reshape(npoints, 2)

In [ ]:
spbd = SpatialBirthAndDeath(dens, papangelou, phi_star_rng, phi_star_dens, c_star)

chains = spbd.run(rng, 10000, 10000, init_state)

In [ ]:
nums = np.array([x.shape[0] for x in chains])
plt.plot(np.arange(len(nums)), nums)

In [ ]:
index = 101
plt.scatter(chains[index][:, 0], chains[index][:, 1])

## Coupling From the Past

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.metrics import pairwise_distances

from pp_mix.simulation.cftp import SimpleDominatedCftp, PerfectStrauss

In [ ]:
beta = 10
gamma = 0.3
R = 0.1
scale = 0.2


def papangelou(csi, x, log=True):
    if x.shape[0] == 0:
        out = 0
    else:
        dists = pairwise_distances(x, csi.reshape(1, -1))
        out = np.log(gamma) * np.sum(dists <= R)
    if not log:
        out = np.exp(out)
    return out

def phi_star_rng():
    return np.random.uniform(size=2)

def phi_star_dens(csi, log=True):
    out = 0;
    if not log:
        out = np.exp(out)
    return out

def dens(x, log=True):
    dists = pairwise_distances(x)
    out = np.sum(np.log(beta) * x.shape[1])
    out += np.log(gamma) * np.sum(dists <= R)
    if not log:
        out = np.exp(out)
    return out

c_star = beta

In [ ]:
cftp = SimpleDominatedCftp(papangelou, phi_star_rng, phi_star_dens, 7, 1)
out = cftp.run_one()
if out is not None:
    plt.scatter(out[:, 0], out[:, 1])

In [ ]:
cftp = PerfectStrauss(papangelou, phi_star_rng, phi_star_dens, 5, 1)
out = cftp.run()
if out is not None:
    plt.scatter(out[:, 0], out[:, 1])

In [ ]:
ranges = np.array([[0, 1], [2, 5], [0, 10]])

In [ ]:
a = np.random.uniform(size=(10, 3))
(a + ranges[:, 0]) * np.diff(ranges, axis=1).reshape(3,)

In [ ]:
np.prod(np.diff(ranges, axis=1).reshape(3,))

In [ ]:
a = [1, 2, 3]
a.remove(1)

In [38]:
import numpy as np
from scipy.linalg import cho_solve, cho_factor
from scipy.stats import multivariate_normal
from math import pi

def multi_normal_lpdf(x, mu, sigma):
    sig_chol = cho_factor(sigma)
    logdet = 2 * np.sum(np.log(sig_chol[0].diagonal()))
    x_min_mu = x - mu
    if (x.shape != mu.shape):
        sols = [cho_solve(sig_chol, x_min_mu[i, :]) for i in range(x.shape[0])]
    else:
        sols = [cho_solve(sig_chol, x_min_mu)]

    out = -0.5 * (np.sum(x_min_mu * sols, axis=1) + logdet + len(mu) * np.log(2 * pi))
    return out

In [39]:
x = np.ones(2)
mu = np.array([0, 0])
sigma = np.array([[2, 0.3], [0.3, 2]])

print(multi_normal_lpdf(x, mu, sigma))
print(multivariate_normal.logpdf(x, mu, sigma))

[-2.95442836]
-2.954428362103635


In [28]:
sig_chol = cho_factor(sigma)
logdet = np.sum(np.log(sig_chol[0].diagonal()))

x_min_mu = x - mu
sol = cho_solve(sig_chol, x_min_mu)
np.dot(x_min_mu.T, sol)

0.8695652173913044

In [37]:
2 * logdet

1.3635373739972745

In [33]:
np.log(np.linalg.det(sigma))

1.3635373739972745

In [31]:
np.sum(x_min_mu * [sol], axis=1)

array([0.86956522])

In [27]:
np.dot(x_min_mu, np.dot(np.linalg.inv(sigma), x_min_mu))

0.8695652173913044